In [2]:
from hstrader import HsTrader
from hstrader.models import Resolution
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
from dotenv import load_dotenv


In [3]:
# Load environment variables from a .env file
load_dotenv()

# Get the CLIENT_ID from the environment variables
id = os.getenv('CLIENT_ID')

# Get the CLIENT_SECRET from the environment variables
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)


In [16]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol=client.get_symbol('EURUSD')
data=client.get_market_history(symbol=symbol.id,resolution=Resolution.H1)

# Create a DataFrame from the retrieved data
df=pd.DataFrame([bar.model_dump() for bar in data])
df.set_index('time',inplace=True)

In [17]:
# Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
nine_period_high = df['high'].rolling(window= 9).max()
nine_period_low = df['low'].rolling(window= 9).min()
df['tenkan-sen'] = (nine_period_high + nine_period_low) /2

# Kijun-sen (Base Line): (26-period high + 26-period low)/2))
twenty_six_period_high=df['high'].rolling(window= 26).max()
twenty_six_period_low =df['low'].rolling(window= 26).max()
df['Kijun-sen']= (twenty_six_period_high+twenty_six_period_low)/2

# The most current closing price plotted 26 time periods behind
df['chikou span']=df['close'].shift(-26)

# Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
df['senkou span A']=((df['tenkan-sen']+df['Kijun-sen'])/2).shift(26)

# Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
Fifty_two_perio_high = df['high'].rolling(window=52).max()
Fifty_two_perio_low = df['low'].rolling(window=52).min()
df['senkou span B']=((Fifty_two_perio_high+Fifty_two_perio_low)/2).shift(26)





In [18]:
# Create a candlestick chart using Plotly
candle=go.Candlestick(x=df.index,open=df['open'].shift(-26),high=df['high'].shift(-26),
                      low=df['low'].shift(-26),close=df['close'].shift(-26),name='Candlestick')

In [19]:
# Define a function to determine the fill color for the Ichimoku Cloud
def get_fill_color(label):
    # Green color for bullish signal, red for bearish
    return 'rgba(0,255,0,0.4)' if label == 1 else 'rgba(255,0,0,0.4)'

# Initialize a new figure for plotting
fig = go.Figure()

# Copy the dataframe to avoid modifying the original data
df1 = df.copy()

# Create a 'label' column to identify bullish (1) and bearish (0) areas
df1['label'] = np.where(df1['senkou span A'] > df1['senkou span B'], 1, 0)

# Group the data by consecutive labels to create continuous areas
df1['group'] = df1['label'].ne(df1['label'].shift()).cumsum()
groups = df1.groupby('group')

# Split the dataframe into subframes for each group
dfs = [data for _, data in groups]

# Plot each group's Ichimoku Cloud area
for df in dfs:
    # Add the Senkou Span A line (invisible, used for filling color)
    fig.add_trace(go.Scatter(x=df.index, y=df['senkou span A'], 
                             line=dict(color='rgba(0,0,0,0)'), name='cloud', showlegend=False))
    # Add the Senkou Span B line and fill the area between them
    fig.add_trace(go.Scatter(x=df.index, y=df['senkou span B'], 
                             line=dict(color='rgba(0,0,0,0)'), fill='tonexty', 
                             fillcolor=get_fill_color(df['label'].iloc[0]), name='cloud', showlegend=False))

# Plot the Kijun-sen line
Kijun_sen = go.Scatter(x=df1.index, y=df1['Kijun-sen'].shift(-26), 
                       line=dict(color='red', width=1), name='Kijun-sen')

# Plot the Tenkan-sen line
tenkan_sen = go.Scatter(x=df1.index, y=df1['tenkan-sen'].shift(-26), 
                        line=dict(color='blue', width=1), name='Tenkan-sen')

# Plot the Chikou Span line
chikou_span = go.Scatter(x=df1.index, y=df1['chikou span'].shift(-26), 
                         line=dict(color='green', width=1), name='Chikou span')

# Plot the Senkou Span A line
senkou_span_A = go.Scatter(x=df1.index, y=df1['senkou span A'], 
                           line=dict(color='green', width=1), name='Senkou span A')

# Plot the Senkou Span B line
senkou_span_B = go.Scatter(x=df1.index, y=df1['senkou span B'], 
                           line=dict(color='red', width=1), name='Senkou span B')

# Add the candlestick trace
fig.add_trace(candle)

# Add all the Ichimoku lines to the figure
fig.add_trace(Kijun_sen)
fig.add_trace(tenkan_sen)
fig.add_trace(chikou_span)
fig.add_trace(senkou_span_A)
fig.add_trace(senkou_span_B)

# Update the layout of the figure
fig.update_layout(width=1100, height=400, xaxis_rangeslider_visible=False, showlegend=True)

# Display the figure
fig.show()


### Example Plot

Below is a static image of the Ichimoku Cloud applied to EUR/USD historical data:

![Ichimoku Plot](img/newplot.png)
